In [ ]:
import os
import flopy as fp
import flopy.utils as fu
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

### load in the voronoi version of the class project model

In [ ]:
modelpath = Path('temp/voronoi/')

In [ ]:
base_sim = fp.mf6.MFSimulation.load(sim_ws=str(modelpath))

In [ ]:
base_m = base_sim.get_model()

### let's make sure we can read in the SFR observations data which we will need for calculating depletion

In [ ]:
sfr_base_obs = pd.read_csv(modelpath / 'sfr_obs.csv', index_col=0)

In [ ]:
sfr_base_obs = sfr_base_obs[['GAGE1','GAGE2']]

In [ ]:
sfr_base_obs

## find all the locations of cells

In [ ]:
cells = pd.DataFrame.from_records(base_m.dis.cell2d.array)
cells

# find the locations of cells that  contain streams and let's not place wells in or under them 

In [ ]:
sfrcells = pd.DataFrame.from_records(base_m.sfr.packagedata.array)
sfrcells

In [ ]:
# make a new columns with just the cellid regardless of layer
sfrcells['cell_no_layer'] = [i[1] for i in sfrcells.cellid]
sfrcells

## now we can find the cells from disv that are _not_ also in SFR using a `set` operation

In [ ]:
len(set(cells.icell2d)), len(set(sfrcells.cell_no_layer)),

In [ ]:
len(set(cells.icell2d))-len(set(sfrcells.cell_no_layer))

In [ ]:
cells_for_wells = list(set(cells.icell2d)-set(sfrcells.cell_no_layer))
len(cells_for_wells)

### now lets load the well file to find a representative pumping rate

In [ ]:
wells = pd.DataFrame.from_records(base_m.wel.stress_period_data.array[0])
wells

In [ ]:
newq = wells.q.mean()
newq

## now let's see about adding a new well - we will reload the model and reset the path to be sure and not stomp on the original model

In [ ]:
sim = fp.mf6.MFSimulation.load(sim_ws=str(modelpath))

In [ ]:
testmod = sim.get_model()

In [ ]:
wells.dtypes

### now we can add a new well with the average Q in layer 0, cell 0

In [ ]:
twell = fp.mf6.ModflowGwfwel(testmod, save_flows=True, stress_period_data=[[(0,0), newq]])

In [ ]:
runpath = Path('temp/depletion/')
sim.set_sim_path(str(runpath))

In [ ]:
sim.exe_name='mf6'
sim.write_simulation()

In [ ]:
sim.run_simulation()

### now read in the SFR observations and compare with the base run

In [ ]:
sfr_test1 = pd.DataFrame.from_records(testmod.sfr.output.obs().get_data())

In [ ]:
sfr_test1

In [ ]:
sfr_test = pd.read_csv(runpath / 'sfr_obs.csv')[['GAGE1','GAGE2']]

In [ ]:
sfr_test

In [ ]:
sfr_base_obs

In [ ]:
(sfr_base_obs.GAGE2.values-sfr_test.GAGE2.values)

# Cool! We can see a difference in flow pretty close to the new pumping rate. Now we are ready to systematically work with the entire model!

## now let's make a dataframe to hold the depletion calculation results

In [ ]:
cellids = np.hstack((cells.icell2d.values, cells.icell2d.values, cells.icell2d.values)) # three layers
layers = np.hstack(([0 for i in range(len(cells.icell2d.values))],
                   [1 for i in range(len(cells.icell2d.values))],
                   [2 for i in range(len(cells.icell2d.values))]))

In [ ]:
cells_with_layers = list(zip(layers,cellids))

In [ ]:
depletion_results = pd.DataFrame(index=pd.MultiIndex.from_tuples(cells_with_layers), data = {'Gage1':np.nan, 'Gage2':np.nan})
depletion_results

# now make a function to run ... the slow way

## we know how to read in the model with flopy, make a change, run it, etc. all using flopy

In [ ]:
def run_a_well_flopy(layer, cellid, newq, sfr_base_obs, depletion_results):
    # read in the simulation
    sim = fp.mf6.MFSimulation.load(sim_ws=str(modelpath))
    # get the model
    testmod = sim.get_model()
    # add the test well 
    well_cellid = (layer,cellid)
    twell = fp.mf6.ModflowGwfwel(testmod, save_flows=True, stress_period_data=[[well_cellid, newq]])
    runpath = Path('temp/depletion/')
    sim.set_sim_path(str(runpath))
    sim.exe_name='mf6'
    sim.write_simulation()
    sim.run_simulation()
    sfr_test = pd.read_csv(runpath / 'sfr_obs.csv')[['GAGE1','GAGE2']]
    depletion_results.loc[well_cellid, 'Gage1'] = (sfr_base_obs.GAGE1.values-sfr_test.GAGE1.values)/newq
    depletion_results.loc[well_cellid, 'Gage2'] = (sfr_base_obs.GAGE2.values-sfr_test.GAGE2.values)/newq
    

In [ ]:
%%timeit -r 5 # the timeit special function will run the folling code a few times and indicate how long 
              # the mean runtime and variance are
for clay in range(3):
    for cellid in cells_for_wells[:5]:
        run_a_well_flopy(clay, cellid, newq,sfr_base_obs, depletion_results)

# now make a function to run ... the FAST way
### we will use flopy only to make the initial model and then manipulate and run only with MF6 files directly

### make a test directory to run in

In [ ]:
sim = fp.mf6.MFSimulation.load(sim_ws=str(modelpath))
# get the model
testmod = sim.get_model()
# add the test well 
twell = fp.mf6.ModflowGwfwel(testmod, save_flows=True, stress_period_data=[[(0,0), newq]])
runpath = Path('temp/depletion/')
sim.set_sim_path(str(runpath))
sim.exe_name='mf6'
sim.write_simulation()

### navigate the wel file to see where to put the new cell ids 

In [ ]:
inwell = open(runpath / 'project_0.wel', 'r').readlines()
inwell

In [ ]:
well_template = []
for line in inwell:
    if str(int(newq)) not in line: # we can just look for our new flowrate
        well_template.append(line)
    else:
        well_template.append('<replace_me>\n')

In [ ]:
well_template = ' '.join(well_template)

In [ ]:
well_template.replace('<replace_me>',f'1 1 {newq:0.4f}')

### now we can run these by just rewriting the wel file directly and suppressing output - should be faster

In [ ]:
# set the current working directory so that we can change as needed to run MF6
cwd = os.getcwd()

In [ ]:
def run_a_well_just_mf6(layer, cellid, newq, sfr_base_obs, depletion_results_fast):
    print(f'running layer = {layer}, cellid = {cellid}\r', end='')
    with open(runpath / 'project_0.wel','w') as ofp:
        ofp.write(well_template.replace('<replace_me>',f'{layer+1:d} {cellid+1:d} {newq:0.4f}'))
    well_cellid = (layer, cellid)
    os.chdir(runpath)
    os.system('mf6 > nul')
    os.chdir(cwd)
    sfr_test = pd.read_csv(runpath / 'sfr_obs.csv')[['GAGE1','GAGE2']]
    depletion_results_fast.loc[well_cellid, 'Gage1'] = (sfr_base_obs.GAGE1.values-sfr_test.GAGE1.values)/newq
    depletion_results_fast.loc[well_cellid, 'Gage2'] = (sfr_base_obs.GAGE2.values-sfr_test.GAGE2.values)/newq

In [ ]:
depletion_results_fast = pd.DataFrame(index=pd.MultiIndex.from_tuples(cells_with_layers), 
                                      data = {'Gage1':np.nan, 'Gage2':np.nan})

In [ ]:
%%timeit -r 5
for lay in range(3):
    for cid in cells_for_wells[:5]:
        run_a_well_just_mf6(lay, cid, newq, sfr_base_obs, depletion_results_fast)

In [ ]:
freshrun = False
if freshrun:
    depletion_results_fast.to_csv('data/depletion_results/depletion_results.csv')

In [ ]:
drf = pd.read_csv('data/depletion_results/depletion_results.csv', index_col=[0,1])

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(9,8))
ax = ax.ravel()
ax_num = 0
for gage_num in range(2):
    for lay_num in range(3):
        cax = ax[ax_num]
        data = drf.loc[lay_num][f'Gage{gage_num+1}'].values
        data[data<0] =0
        mm = fp.plot.PlotMapView(model= base_m)
        cb = mm.plot_array(data, vmin=0, vmax=1.0, ax=cax, cmap='magma')
        mm.plot_grid(lw=0.5, color="0.25",ax=cax)
        cax.set_aspect('equal')
        cax.set_title(f'Gage{gage_num+1} layer {lay_num}')
        plt.colorbar(cb, ax=cax)
        cax.axis('off')
        ax_num += 1